<a href="https://colab.research.google.com/github/danyssilva/ANN/blob/main/Dataset_IMDB_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarefa 4:

Use a classe pipeline da biblioteca transformers do HuggingFace (Behind the pipeline - Hugging Face NLP Course) para analisar o sentimento da base IMBD "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz."

In [1]:
# Baixando aqrquivo e descompactando
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
!cat aclImdb/train/neg/8487_1.txt
# pos/4077_10.txt
# neg/8487_1.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  8416k      0  0:00:09  0:00:09 --:--:-- 11.9M
I haven't seen anything this bad since I walked out of the James Bond movie "Moonraker" twenty years ago. I managed to sit through the entirety of this one only because of Tilda Swinton, but there was nothing she could do to save this beast.<br /><br />As a cross between "Pi", "Orlando", and "Tron", this movie failed miserably in every aspect of moviemaking. The characters were cardboard and unable to evoke any kind of sympathy. The plot was wholly unbelievable. The acting was, with the exception of Swinton, amateur. The computer graphics were worse than in "Tron." Timothy Leary was extremely annoying. I could go on, but what's the point.<br /><br />The only good thing I can say about this film is that Tilda Swinton was in it. I have no idea why an actr

In [2]:
import os, pathlib, shutil, random
base_dir = pathlib.Path("aclImdb")
train_dir = base_dir / "train"
val_dir = base_dir / "val"
test_dir = base_dir / "test"
for category in ["pos", "neg"]:
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname, val_dir / category / fname)

In [4]:
from tensorflow import keras
batch_size = 32
train_dataset = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size)
val_dataset = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size)
test_dataset = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [5]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="textattack/bert-base-uncased-imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [6]:
pipe("this movie failed miserably")

[{'label': 'LABEL_0', 'score': 0.9977490305900574}]

In [7]:
from sklearn.metrics import accuracy_score
import numpy as np
import time

In [8]:
ypred = []
test_labels = []
start = time.time()
for batch, (inputs, targets) in enumerate(test_dataset):
  for i in range(len(inputs)):
    yp = pipe(inputs[i].numpy().decode('UTF-8')[:512])
    test_labels.append(int(targets[i].numpy()==1))
    ypred.append(int(yp[0]['label']=="LABEL_1"))
  print(batch, time.time()-start, accuracy_score(test_labels, ypred))

0 17.414193630218506 0.875
1 32.3302059173584 0.890625
2 47.95418953895569 0.875
3 68.05886650085449 0.859375
4 83.84032201766968 0.85625
5 102.59314012527466 0.8697916666666666
6 119.54949617385864 0.8705357142857143
7 135.36669898033142 0.86328125
8 151.03321886062622 0.8611111111111112
9 167.4294171333313 0.8625
10 184.7362117767334 0.8664772727272727
11 201.2559723854065 0.8671875
12 216.5656304359436 0.875
13 233.9732882976532 0.8727678571428571
14 251.0556275844574 0.875
15 267.78502678871155 0.875
16 283.4046120643616 0.875
17 299.7765803337097 0.875
18 316.8488960266113 0.8766447368421053
19 333.2210216522217 0.8765625
20 349.6468765735626 0.8764880952380952
21 367.25961661338806 0.8735795454545454
22 384.21792459487915 0.875
23 400.07255029678345 0.875
24 416.214453458786 0.875
25 433.39838910102844 0.8774038461538461
26 449.55972385406494 0.8784722222222222
27 465.8175449371338 0.8772321428571429
28 482.05216431617737 0.8793103448275862
29 499.1608431339264 0.8802083333333334